In [1]:
!pip install transformers tensorflow

In [2]:
# ---- LOAD DATASET INTO COLAB ----
from google.colab import files
import zipfile
import os

print("Upload your dataset ZIP or CSV files:")
uploaded = files.upload()

os.makedirs("datasets", exist_ok=True)

for fname in uploaded.keys():
    # If it's a ZIP → extract inside datasets/
    if fname.endswith(".zip"):
        print("Extracting", fname)
        with zipfile.ZipFile(fname, "r") as z:
            z.extractall("datasets")
        print("Done.")
    else:
        # Normal CSV → move to datasets/
        os.rename(fname, f"datasets/{fname}")

print("Dataset is ready at /content/datasets/")


Upload your dataset ZIP or CSV files:


Saving train.raw to train.raw
Dataset is ready at /content/datasets/


In [11]:
# Install dependencies if running in Colab
# !pip install transformers tensorflow

import os
import csv
import random
import json
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from datetime import datetime

# --------------------------
# CONFIG / HYPERPARAMETERS
# --------------------------
DATA_ROOT = os.path.abspath(os.path.join(os.getcwd(), "datasets"))
MODEL_NAME = "bert-base-uncased"
SEED = 42
MAX_EXAMPLES = 2048
RESULTS_FILE = "tuning_results.jsonl"

GRID = [
    {"batch_size": 8,  "lr": 2e-5, "epochs": 2, "max_len": 64, "window": 2, "gcn_hidden": 128},
    {"batch_size": 16, "lr": 3e-5, "epochs": 2, "max_len": 64, "window": 2, "gcn_hidden": 128},
    {"batch_size": 8,  "lr": 2e-5, "epochs": 3, "max_len": 96, "window": 3, "gcn_hidden": 128}
]

tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# --------------------------
# DATA LOADER
# --------------------------
def load_texts_from_folder(data_root):
    texts, labels = [], []
    candidates = [os.path.join(data_root, "headlines"),
                  os.path.join(data_root, "riloff"),
                  data_root]

    for c in candidates:
        if not os.path.exists(c):
            continue
        for fname in os.listdir(c):
            if not fname.lower().endswith(".csv"):
                continue
            p = os.path.join(c, fname)
            try:
                with open(p, encoding="utf-8") as f:
                    reader = csv.reader(f)
                    header = next(reader, None)
                    for row in reader:
                        if not row:
                            continue
                        text = row[0].strip()
                        if not text:
                            continue
                        label = int(row[1]) if len(row) > 1 and row[1].isdigit() else 0
                        texts.append(text)
                        labels.append(label)
            except Exception:
                continue

    if len(texts) == 0:
        # fallback small dataset
        texts = [
            "I absolutely love being ignored.",
            "That was a fantastic performance.",
            "Yeah right, I really wanted more spam emails.",
            "He did a wonderful job.",
            "Oh great, another rainy day.",
            "She nailed it, truly impressive.",
            "I adore waiting in long lines.",
            "That was the best movie I've ever not watched."
        ]
        labels = [1, 0, 1, 0, 1, 0, 1, 1]

    return texts, labels

# --------------------------
# ADJACENCY MATRIX BUILDER
# --------------------------
def build_adj_and_norm(N, L, window, att_mask_np):
    A = np.zeros((N, L, L), dtype=np.float32)
    for i in range(N):
        valid_positions = np.where(att_mask_np[i] == 1)[0]
        for t in valid_positions:
            start = max(0, t - window)
            end = min(L - 1, t + window)
            A[i, t, start:end+1] = 1.0

    A_norm = np.zeros_like(A)
    for i in range(N):
        Ai = A[i]
        D = Ai.sum(axis=1)
        D_inv_sqrt = np.zeros_like(D)
        mask = D > 0
        if mask.any():
            D_inv_sqrt[mask] = 1.0 / np.sqrt(D[mask])
        D_mat = np.diag(D_inv_sqrt)
        A_norm[i] = D_mat.dot(Ai).dot(D_mat)
    return A_norm

# --------------------------
# GCN LAYER
# --------------------------
class GCNLayer(tf.keras.layers.Layer):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.W = self.add_weight(shape=(in_dim, out_dim),
                                 initializer="glorot_uniform",
                                 trainable=True)

    def call(self, A, X):
        AX = tf.matmul(A, X)
        return tf.matmul(AX, self.W)

# --------------------------
# BERT + 4-LAYER GCN MODEL
# --------------------------
class BERT_GCN(tf.keras.Model):
    def __init__(self, bert_model, gcn_hidden, num_classes=2):
        super().__init__()
        self.bert = bert_model

        hidden_dim = gcn_hidden
        # Alternating 4 layers
        self.gcn1 = GCNLayer(768, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, hidden_dim)
        self.gcn3 = GCNLayer(hidden_dim, hidden_dim)
        self.gcn4 = GCNLayer(hidden_dim, hidden_dim)

        self.classifier = tf.keras.layers.Dense(num_classes)

    def call(self, inputs, training=False):
        input_ids, attention_mask, adj = inputs

        bert_out = self.bert(input_ids, attention_mask=attention_mask, training=training)
        X = bert_out.last_hidden_state   # (B, L, 768)

        H1 = tf.nn.relu(self.gcn1(adj, X))
        H2 = tf.nn.relu(self.gcn2(adj, H1))
        H3 = tf.nn.relu(self.gcn3(adj, H2))
        H4 = tf.nn.relu(self.gcn4(adj, H3))

        mask = tf.cast(tf.expand_dims(attention_mask, -1), tf.float32)
        sum_h = tf.reduce_sum(H4 * mask, axis=1)
        denom = tf.reduce_sum(mask, axis=1)
        graph_repr = sum_h / (denom + 1e-9)

        logits = self.classifier(graph_repr)
        return logits

# --------------------------
# TRAINING / TUNING LOOP
# --------------------------
texts, labels = load_texts_from_folder(DATA_ROOT)
if len(texts) > MAX_EXAMPLES:
    idxs = list(range(len(texts)))
    random.shuffle(idxs)
    idxs = idxs[:MAX_EXAMPLES]
    texts = [texts[i] for i in idxs]
    labels = [labels[i] for i in idxs]

open(RESULTS_FILE, "a").close()

for cfg in GRID:
    TIMESTAMP = datetime.now().strftime("%Y%m%d-%H%M%S")
    exp_name = f"exp_{TIMESTAMP}_bs{cfg['batch_size']}_lr{cfg['lr']}_epochs{cfg['epochs']}_len{cfg['max_len']}_win{cfg['window']}_hid{cfg['gcn_hidden']}"
    print("\n=== RUN:", exp_name, "===\n")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    bert = TFAutoModel.from_pretrained(MODEL_NAME, from_pt=True)

    enc = tokenizer(texts, padding=True, truncation=True, max_length=cfg["max_len"], return_tensors="tf")
    input_ids, attention_mask = enc["input_ids"], enc["attention_mask"]
    labels_tf = tf.constant(labels, dtype=tf.int32)
    N, L = input_ids.shape

    adj_norm = build_adj_and_norm(N, L, cfg["window"], attention_mask.numpy())
    adj_norm = tf.constant(adj_norm, dtype=tf.float32)

    # split
    indices = np.arange(N)
    np.random.shuffle(indices)
    train_size = int(0.8 * N) if N > 1 else N
    train_idx, val_idx = indices[:train_size], indices[train_size:]

    def sel(x, ids):
        return tf.gather(x, ids) if ids.size > 0 else tf.zeros((0,) + x.shape[1:], dtype=x.dtype)

    train_input_ids = sel(input_ids, train_idx)
    train_attention_mask = sel(attention_mask, train_idx)
    train_adj = sel(adj_norm, train_idx)
    train_labels = sel(labels_tf, train_idx)

    val_input_ids = sel(input_ids, val_idx)
    val_attention_mask = sel(attention_mask, val_idx)
    val_adj = sel(adj_norm, val_idx)
    val_labels = sel(labels_tf, val_idx)

    train_ds = tf.data.Dataset.from_tensor_slices((train_input_ids, train_attention_mask, train_adj, train_labels))
    train_ds = train_ds.shuffle(512, seed=SEED).batch(cfg["batch_size"]).prefetch(tf.data.AUTOTUNE)

    has_val = val_input_ids.shape[0] > 0
    if has_val:
        val_ds = tf.data.Dataset.from_tensor_slices((val_input_ids, val_attention_mask, val_adj, val_labels))
        val_ds = val_ds.batch(cfg["batch_size"]).prefetch(tf.data.AUTOTUNE)
    else:
        val_ds = None

    model = BERT_GCN(bert, cfg["gcn_hidden"])
    model.compile(optimizer=tf.keras.optimizers.Adam(cfg["lr"]),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])

    # warmup build
    if train_input_ids.shape[0] > 0:
        _sample = (train_input_ids[:1], train_attention_mask[:1], train_adj[:1])
        _ = model(_sample, training=False)

    # checkpoint
    ckpt_dir = os.path.join("runs", exp_name)
    os.makedirs(ckpt_dir, exist_ok=True)
    ckpt_path = os.path.join(ckpt_dir, "best_weights.weights.h5")
    callbacks = [tf.keras.callbacks.ModelCheckpoint(
        ckpt_path,
        save_weights_only=True,
        save_best_only=True,
        monitor="val_accuracy" if has_val else "loss",
        mode="max" if has_val else "min",
        verbose=1
    )]

    # fit
    history = model.fit(train_ds.map(lambda ids, att, A, y: ((ids, att, A), y)),
                        validation_data=(val_ds.map(lambda ids, att, A, y: ((ids, att, A), y)) if has_val else None),
                        epochs=cfg["epochs"],
                        callbacks=callbacks,
                        verbose=1)

    # evaluation
    if has_val:
        val_loss, val_acc = map(float, model.evaluate(val_ds.map(lambda ids, att, A, y: ((ids, att, A), y)), verbose=0))
    else:
        val_loss, val_acc = None, None

    # save final
    model.save_weights(os.path.join(ckpt_dir, "final_weights.weights.h5"))
    tokenizer.save_pretrained(ckpt_dir)

    # save results
    result = {"exp": exp_name, "config": cfg, "val_loss": val_loss, "val_acc": val_acc,
              "history": {k: [float(x) for x in v] for k, v in history.history.items()},
              "ckpt_dir": ckpt_dir}
    with open(RESULTS_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(result) + "\n")

    print("Completed", exp_name, "val_acc:", val_acc, "ckpt:", ckpt_dir)

print("\nTUNING FINISHED. Results appended to", RESULTS_FILE)



=== RUN: exp_20251211-144556_bs8_lr2e-05_epochs2_len64_win2_hid128 ===



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Epoch 1/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.6667 - loss: 0.7012
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to runs/exp_20251211-144556_bs8_lr2e-05_epochs2_len64_win2_hid128/best_weights.weights.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.6667 - loss: 0.7012 - val_accuracy: 0.5000 - val_loss: 0.6637
Epoch 2/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step - accuracy: 0.6667 - loss: 0.6647
Epoch 2: val_accuracy did not improve from 0.50000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 632ms/step - accuracy: 0.6667 - loss: 0.6647 - val_accuracy: 0.5000 - val_loss: 0.6630
Completed exp_20251211-144556_bs8_lr2e-05_epochs2_len64_win2_hid128 val_acc: 0.5 ckpt: runs/exp_20251211-144556_bs8_lr2e-05_epochs2_len64_win2_hid128

=== RUN: exp_20251211-144626_bs16_lr3e-05_epochs2_len64_win2_hid128 ===



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Epoch 1/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.5000 - loss: 0.6943
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to runs/exp_20251211-144626_bs16_lr3e-05_epochs2_len64_win2_hid128/best_weights.weights.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.5000 - loss: 0.6943 - val_accuracy: 0.5000 - val_loss: 0.7162
Epoch 2/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.5000 - loss: 0.6751
Epoch 2: val_accuracy did not improve from 0.50000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step - accuracy: 0.5000 - loss: 0.6751 - val_accuracy: 0.5000 - val_loss: 0.7149
Completed exp_20251211-144626_bs16_lr3e-05_epochs2_len64_win2_hid128 val_acc: 0.5 ckpt: runs/exp_20251211-144626_bs16_lr3e-05_epochs2_len64_win2_hid128

=== RUN: exp_20251211-144657_bs8_lr2e-05_epochs3_len96_win3_hid128 ===



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.1667 - loss: 0.7316
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to runs/exp_20251211-144657_bs8_lr2e-05_epochs3_len96_win3_hid128/best_weights.weights.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.1667 - loss: 0.7316 - val_accuracy: 1.0000 - val_loss: 0.6612
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.1667 - loss: 0.7233
Epoch 2: val_accuracy did not improve from 1.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.1667 - loss: 0.7233 - val_accuracy: 1.0000 - val_loss: 0.6616
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step - accuracy: 0.6667 - loss: 0.6848
Epoch 3: val_accuracy did not improve from 1.00000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 668ms/step - accuracy: 0.6667 - loss: 0.6848 - val_accuracy: 1.0000 - val_loss: 0.6621
Completed exp_20251211-144657_bs8_lr2e-05_epochs3_len96_win3_hid128 val_acc: 1.0 ckpt: runs/exp_20251211-144657_bs8_lr2e-05_epochs3_len96_win3_hid128

TUNING

In [13]:
# --------------------------
# USE PART OF TRAINING DATA FOR QUICK TEST
# --------------------------
BATCH_SIZE = 8

# select a small subset (e.g., first 20 examples)
test_input_ids = train_input_ids[:20]
test_attention_mask = train_attention_mask[:20]
test_adj = train_adj[:20]
test_labels_tf = train_labels[:20]

test_ds = tf.data.Dataset.from_tensor_slices(
    (test_input_ids, test_attention_mask, test_adj, test_labels_tf)
).batch(BATCH_SIZE)

# --------------------------
# EVALUATION
# --------------------------
loss, acc = model.evaluate(
    test_ds.map(lambda ids, att, A, y: ((ids, att, A), y)), verbose=1
)
print(f"Quick Test Loss: {loss:.4f}, Quick Test Accuracy: {acc:.4f}")

# --------------------------
# PREDICTIONS & METRICS
# --------------------------
preds = []
for batch in test_ds:
    b_ids, b_att, b_adj, _ = batch
    logits = model((b_ids, b_att, b_adj), training=False)
    batch_preds = tf.argmax(logits, axis=1).numpy().tolist()
    preds.extend(batch_preds)

print("Classification Report (subset of train):")
print(classification_report(test_labels_tf.numpy(), preds))
print("Confusion Matrix (subset of train):")
print(confusion_matrix(test_labels_tf.numpy(), preds))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 860ms/step - accuracy: 0.1667 - loss: 0.7151
Quick Test Loss: 0.7151, Quick Test Accuracy: 0.1667
Classification Report (subset of train):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.25      0.33      0.29         3

    accuracy                           0.17         6
   macro avg       0.12      0.17      0.14         6
weighted avg       0.12      0.17      0.14         6

Confusion Matrix (subset of train):
[[0 3]
 [2 1]]
